## Сучасні технології програмування в системах зі штучним інтелектом. Практична робота 6. **Робота згорткової нейронної мережі (CNN) у Tensorflow**
## Виконав студент групи ШІДМ-51 Тертишний Владислав

## Convolution (вбудований фільтр)

Знайшовши ці важливі функції, ми можемо залишити небажані пікселі без шкоди для якості результату. За допомогою цього методу ми можемо надати моделі людський рівень розпізнавання зображення в реальному світі. Отже, для цього ми маємо згортку.

Згортка є найбільш заплутаною та найскладнішою темою в Інтернеті, але це просто пошук зображення шляхом ковзання фільтра (ядра) по зображенню, щоб знайти різні функції зображення. Ядра — це просто двовимірні матриці з різними вагами в них. По суті, це ядро ​​передаватиме зображення, замінюючи значення пікселів середнім значенням суми його ваги у відповідній частині зображення. Ці ядра — чудовий спосіб знайти найважливіші функції зображення.

Ми застосуємо кілька випадково згенерованих ядер до зображення, щоб знайти багато різних функцій зображень  

Тож після застосування цього шару згортки до нашої моделі нам потрібно об’єднати функції.







Завантажимо набор зображень рукописних цифр MNIST.


In [7]:
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Plotting random images from dataset

# import matplotlib.pyplot as plt
# import random
# plt.figure(figsize = (12,5))
# for i in range(8):
# ind = random.randint(0, len(X_train))
# plt.subplot(240+1+i)
# plt.imshow(X_train[ind], cmap=True)


Виконаємо попередньу обробку зображень: приведення елементів масиву, які відповідають кожному пікселу зображення, до формату float32 та перекодування елементів масиву з діапазону 0-255 к діапазону 0-1.

Мітки зображень перетворюємо в категоріальний вигляд.

In [8]:
from tensorflow.keras.utils import to_categorical

# convert image datatype from integers to floats
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# normalising piel values
X_train = X_train/255.0
X_test = X_test/255.0

# reshape images to add channel dimension
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

# One-hot encoding label
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

model = Sequential()

# Layer 1
# Conv 1
model.add(Conv2D(filters=6, kernel_size=(5, 5), strides=1, activation = 'relu', input_shape = (28,28,1)))
# Pooling 1
model.add(MaxPooling2D(pool_size=(2, 2), strides = 2))

# Layer 2
# Conv 2
model.add(Conv2D(filters=16, kernel_size=(5, 5), strides=1, activation='relu'))
# Pooling 2
model.add(MaxPooling2D(pool_size = 2, strides = 2))

# Flatten
model.add(Flatten())

# Layer 3
# Fully connected layer 1
model.add(Dense(units=120, activation='relu'))

#Layer 4
#Fully connected layer 2
model.add(Dense(units=84, activation='relu'))

#Layer 5
#Output Layer
model.add(Dense(units=10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 20
batch_size = 512
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
					steps_per_epoch=X_train.shape[0]//batch_size,
					validation_data=(X_test, y_test),
					validation_steps=X_test.shape[0]//batch_size, verbose = 0 )

_, acc = model.evaluate(X_test, y_test, verbose = 0)

## **Завдання**



1.   Перевірити, як впливаєна час і результати навчання моделі кількість епох навчання
2.  Дослідити, як змінить хід і результати навчання зміна кількості нейронів в щільних та конволюційних шарах.
3.   Додати один, а потім два конволюційні блоки, також дослідити якість навчання моделі.
4.   Побудувати аналогічний класифікатор для набору даних Fashion MNIST з попередньої роботи.



In [10]:
import time
import matplotlib.pyplot as plt

# 1. Дослідження впливу кількості епох
print("Завдання 1: Вплив кількості епох на навчання")
epochs_to_test = [5, 10, 20, 30]
epoch_results = []

for epoch_num in epochs_to_test:
    start_time = time.time()
    
    model = Sequential([
        Conv2D(6, (5, 5), activation='relu', input_shape=(28,28,1)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(16, (5, 5), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(120, activation='relu'),
        Dense(84, activation='relu'),
        Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    history = model.fit(X_train, y_train,
                       epochs=epoch_num,
                       batch_size=512,
                       validation_data=(X_test, y_test),
                       verbose=0)
    
    training_time = time.time() - start_time
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    
    print(f'Епохи: {epoch_num}, Час: {training_time:.2f}с, Точність: {accuracy:.4f}')
    epoch_results.append((epoch_num, training_time, accuracy))

# 2. Дослідження різної кількості нейронів
print("\nЗавдання 2: Вплив кількості нейронів")
architectures = [
    {'conv1': 6, 'conv2': 16, 'dense1': 120, 'dense2': 84},  # Оригінальна
    {'conv1': 12, 'conv2': 32, 'dense1': 240, 'dense2': 168},  # Збільшена
    {'conv1': 3, 'conv2': 8, 'dense1': 60, 'dense2': 42}  # Зменшена
]

for arch in architectures:
    model = Sequential([
        Conv2D(arch['conv1'], (5, 5), activation='relu', input_shape=(28,28,1)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(arch['conv2'], (5, 5), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(arch['dense1'], activation='relu'),
        Dense(arch['dense2'], activation='relu'),
        Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    history = model.fit(X_train, y_train,
                       epochs=20,
                       batch_size=512,
                       validation_data=(X_test, y_test),
                       verbose=0)
    
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f'Архітектура {arch}: Точність: {accuracy:.4f}')

# 3. Додавання конволюційних блоків
print("\nЗавдання 3: Додаткові конволюційні блоки")

# Модель з одним додатковим блоком
model_extra1 = Sequential([
    Conv2D(6, (5, 5), activation='relu', input_shape=(28,28,1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(16, (5, 5), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), padding='same'),
    Flatten(),
    Dense(120, activation='relu'),
    Dense(84, activation='relu'),
    Dense(10, activation='softmax')
])

model_extra1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history1 = model_extra1.fit(X_train, y_train, epochs=20, batch_size=512, 
                           validation_data=(X_test, y_test), verbose=0)
_, acc1 = model_extra1.evaluate(X_test, y_test, verbose=0)
print(f'Точність з 1 додатковим блоком: {acc1:.4f}')

# Модель з двома додатковими блоками
model_extra2 = Sequential([
    Conv2D(6, (5, 5), activation='relu', input_shape=(28,28,1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(16, (5, 5), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), padding='same'),
    Flatten(),
    Dense(120, activation='relu'),
    Dense(84, activation='relu'),
    Dense(10, activation='softmax')
])

model_extra2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history2 = model_extra2.fit(X_train, y_train, epochs=20, batch_size=512, 
                           validation_data=(X_test, y_test), verbose=0)
_, acc2 = model_extra2.evaluate(X_test, y_test, verbose=0)
print(f'Точність з 2 додатковими блоками: {acc2:.4f}')

# 4. Fashion MNIST
print("\nЗавдання 4: Fashion MNIST")
from tensorflow.keras.datasets import fashion_mnist

(X_train_fashion, y_train_fashion), (X_test_fashion, y_test_fashion) = fashion_mnist.load_data()

# Попередня обробка даних
X_train_fashion = X_train_fashion.astype('float32') / 255.0
X_test_fashion = X_test_fashion.astype('float32') / 255.0

X_train_fashion = X_train_fashion.reshape(X_train_fashion.shape[0], 28, 28, 1)
X_test_fashion = X_test_fashion.reshape(X_test_fashion.shape[0], 28, 28, 1)

y_train_fashion = to_categorical(y_train_fashion)
y_test_fashion = to_categorical(y_test_fashion)

# Створення та навчання моделі для Fashion MNIST
model_fashion = Sequential([
    Conv2D(6, (5, 5), activation='relu', input_shape=(28,28,1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(16, (5, 5), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(120, activation='relu'),
    Dense(84, activation='relu'),
    Dense(10, activation='softmax')
])

model_fashion.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_fashion = model_fashion.fit(X_train_fashion, y_train_fashion,
                                  epochs=20,
                                  batch_size=512,
                                  validation_data=(X_test_fashion, y_test_fashion),
                                  verbose=0)

_, accuracy_fashion = model_fashion.evaluate(X_test_fashion, y_test_fashion, verbose=0)
print(f'Точність на Fashion MNIST: {accuracy_fashion:.4f}')

Завдання 1: Вплив кількості епох на навчання
Епохи: 5, Час: 15.09с, Точність: 0.9809
Епохи: 10, Час: 29.73с, Точність: 0.9863
Епохи: 20, Час: 58.49с, Точність: 0.9888
Епохи: 30, Час: 87.38с, Точність: 0.9901

Завдання 2: Вплив кількості нейронів
Архітектура {'conv1': 6, 'conv2': 16, 'dense1': 120, 'dense2': 84}: Точність: 0.9887
Архітектура {'conv1': 12, 'conv2': 32, 'dense1': 240, 'dense2': 168}: Точність: 0.9921
Архітектура {'conv1': 3, 'conv2': 8, 'dense1': 60, 'dense2': 42}: Точність: 0.9826

Завдання 3: Додаткові конволюційні блоки
Точність з 1 додатковим блоком: 0.9896
Точність з 2 додатковими блоками: 0.9877

Завдання 4: Fashion MNIST
Точність на Fashion MNIST: 0.8786


На основі отриманих результатів можна зробити наступні висновки:

1. **Вплив кількості епох на навчання:**
- Спостерігається стабільне покращення точності зі збільшенням кількості епох: від 0.9809 (5 епох) до 0.9901 (30 епох)
- Однак приріст точності сповільнюється: між 5 і 10 епохами різниця складає 0.0054, тоді як між 20 і 30 епохами лише 0.0013
- Час навчання зростає лінійно з кількістю епох
- Оптимальним балансом між часом навчання та точністю можна вважати 20 епох, так як подальше збільшення дає незначний приріст точності при суттєвому збільшенні часу навчання

2. **Вплив кількості нейронів:**
- Збільшена архітектура (12-32-240-168) показала найкращий результат з точністю 0.9921
- Зменшена архітектура (3-8-60-42) показала найгірший результат - 0.9826
- Різниця між найкращою та найгіршою архітектурою складає близько 0.01, що свідчить про достатню робастність моделі
- Оригінальна архітектура (6-16-120-84) показала збалансований результат 0.9887

3. **Вплив додаткових конволюційних блоків:**
- Додавання одного конволюційного блоку дещо покращило точність до 0.9896
- Додавання другого блоку призвело до незначного погіршення результату (0.9877)
- Це може свідчити про появу перенавчання при надмірному ускладненні архітектури

4. **Порівняння з Fashion MNIST:**
- Точність на Fashion MNIST (0.8786) значно нижча ніж на звичайному MNIST
- Це очікуваний результат, оскільки Fashion MNIST є складнішим набором даних з більшою варіативністю об'єктів
- Різниця в точності складає приблизно 0.11, що відображає більшу складність класифікації одягу порівняно з цифрами

**Загальні висновки:**
1. Модель показує високу ефективність на наборі MNIST, досягаючи точності понад 99%
2. Оптимальна конфігурація включає:
   - 20 епох навчання
   - Збільшену кількість нейронів
   - Один додатковий конволюційний блок
3. Подальше ускладнення архітектури або збільшення кількості епох дає незначний приріст продуктивності
4. Модель демонструє достатню гнучкість для роботи з різними наборами даних, хоча ефективність залежить від складності задачі
